In [58]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from common_functions import display

spark = (SparkSession.builder.remote("sc://localhost:15002").getOrCreate())

data = [[2021, "CS5302-AI621", "Albany", "M", 42]]
columns = ["Year", "First_Name", "County", "Gender", "Count"]

mydf = spark.createDataFrame(data, schema="Year int, First_Name STRING, County STRING, Gender STRING, Count int")
display(mydf)

Year,First_Name,County,Gender,Count
2021,CS5302-AI621,Albany,M,42


In [59]:
mydf.show()

+----+------------+------+------+-----+
|Year|  First_Name|County|Gender|Count|
+----+------------+------+------+-----+
|2021|CS5302-AI621|Albany|     M|   42|
+----+------------+------+------+-----+



In [60]:
nyc_df = spark.read.csv("/opt/spark/data/nyc_health_data.csv", header=True, inferSchema=True)
nyc_df = nyc_df.withColumnRenamed("Sex", "Gender")
display(nyc_df,5)

Year,First Name,County,Gender,Count
2022,OLIVIA,Albany,F,16
2022,AMELIA,Albany,F,15
2022,AVERY,Albany,F,12
2022,EMMA,Albany,F,11
2022,CHARLOTTE,Albany,F,11


In [61]:
nyc_df.explain("formatted")

== Physical Plan ==
* Project (2)
+- Scan csv  (1)


(1) Scan csv 
Output [5]: [Year#100, First Name#101, County#102, Sex#103, Count#104]
Batched: false
Location: InMemoryFileIndex [file:/opt/spark/data/nyc_health_data.csv]
ReadSchema: struct<Year:int,First Name:string,County:string,Sex:string,Count:int>

(2) Project [codegen id : 1]
Output [5]: [Year#100, First Name#101, County#102, Sex#103 AS Gender#113, Count#104]
Input [5]: [Year#100, First Name#101, County#102, Sex#103, Count#104]




In [62]:
mydf.printSchema()
nyc_df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- First_Name: string (nullable = true)
 |-- County: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Count: integer (nullable = true)

root
 |-- Year: integer (nullable = true)
 |-- First Name: string (nullable = true)
 |-- County: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Count: integer (nullable = true)



In [63]:
# Apache Spark union() method to combine the contents of your first DataFrame mydf with DataFrame nyc_df data loaded from the CSV file.
df = mydf.union(nyc_df)
display(df,5)

Year,First_Name,County,Gender,Count
2021,CS5302-AI621,Albany,M,42
2022,OLIVIA,Albany,F,16
2022,AMELIA,Albany,F,15
2022,AVERY,Albany,F,12
2022,EMMA,Albany,F,11


In [64]:
display(
    df.filter(df["Count"] > 50), 5
    )

Year,First_Name,County,Gender,Count
2022,SOPHIA,Bronx,F,65
2022,LUNA,Bronx,F,65
2022,EMMA,Bronx,F,60
2022,ISABELLA,Bronx,F,55
2022,LIAM,Bronx,M,156


In [67]:
from pyspark.sql.functions import desc
display(
    df.select("First_Name", "Count").orderBy(desc("Count")), 5)

First_Name,Count
DAVID,297
DAVID,287
ETHAN,273
DAVID,273
DAVID,272


In [69]:
subsetDF = df.filter((df["Year"] == 2009) & (df["Count"] > 100) & (df["gender"] == "F")).select("First_Name", "County", "Count").orderBy(desc("Count"))
display(subsetDF, 5)

First_Name,County,Count
LEAH,Kings,186
SARAH,Kings,183
RACHEL,Kings,170
ESTHER,Kings,168
ISABELLA,Queens,165


Spark Transformations (No Execution happens until action is executed (show, count, write, etc.)

In [70]:
# Immutability & Laziness
subset_df = df.filter(df.Count > 100) 

In [72]:
# Column Projection & Derivation
# select keeps or renames columns, withColumn creates derived columns, drop removes fields.

df_a = df.select("First_Name", "County")
df_b = df.withColumn("Count_Double", df.Count * 2).drop("Year")
display(df_b,5)

First_Name,County,Gender,Count,Count_Double
CS5302-AI621,Albany,M,42,84
OLIVIA,Albany,F,16,32
AMELIA,Albany,F,15,30
AVERY,Albany,F,12,24
EMMA,Albany,F,11,22


In [86]:
# groupBy followed by agg computes rollups; use aliases for clarity.

df_grouped = df.groupBy("Year").agg(F.sum("Count").alias("Total_Count"))

In [84]:
# Joins: inner, left, right, full, semi, anti

# inner join: returns only rows whose keys exist in both tables
df_inner = df.join(nyc_df, ["Year", "County"], "inner")

# right join: keeps all rows from nyc_df and matches from df where keys align
df_right = df.join(nyc_df, ["Year", "County"], "right")

# full join: keeps every row from both sides, matching where keys intersect
df_full = df.join(nyc_df, ["Year", "County"], "outer")

# left semi join: keeps rows from df that have matching keys in nyc_df without nyc_df columns
df_left_semi = df.join(nyc_df, ["Year", "County"], "leftsemi")

# left anti join: keeps rows from df whose keys do not appear in nyc_df
df_left_anti = df.join(nyc_df, ["Year", "County"], "leftanti")

# left join: keeps all rows from df and matches from nyc_df where keys align
df_left = df.join(nyc_df, ["Year", "County"], "left")

In [83]:
# Set Operations: unionByName, intersect, and subtract combine DataFrames with aligned schemas.
# unionByName: Aligns columns by name before combining DataFrames, adding nulls for missing fields.
# intersect: Keeps only rows that exist in both DataFrames with identical column values.
# subtract: Emits rows from the first DataFrame that are absent from the second.

df_union = df.unionByName(nyc_df, allowMissingColumns=True)
df_intersect = df.intersect(nyc_df)
df_subtract = df.subtract(nyc_df)

In [82]:
# dropDuplicates targets specific columns
# Rows sharing identical values in both First_Name and County are collapsed to their first occurrence; 
# other column values come from that retained row.

df.dropDuplicates(["First_Name", "County"])

DataFrame[Year: int, First_Name: string, County: string, Gender: string, Count: int]

In [80]:
#Partition Management: repartition increases partitions for parallelism; coalesce reduces without shuffle.
df.repartition("County")

DataFrame[Year: int, First_Name: string, County: string, Gender: string, Count: int]

In [79]:
# Window Transformations - window functions add rankings or moving aggregates without collapsing rows.
from pyspark.sql.window import Window

df_windows = df.withColumn("Rank", F.row_number().over(Window.partitionBy("Year").orderBy(desc("Count"))))
display(df_windows,5)

Year,First_Name,County,Gender,Count,Rank
2007,DANIEL,Kings,M,258,1
2007,DAVID,Kings,M,251,2
2007,JAYDEN,Kings,M,239,3
2007,DANIEL,Queens,M,234,4
2007,JAYDEN,Bronx,M,226,5


In [87]:
# Caching & Checkpoints - cache/persist reuse expensive intermediate plans; checkpoint breaks lineage for stability.
df.cache()

DataFrame[Year: int, First_Name: string, County: string, Gender: string, Count: int]

In [73]:
# Diagnostics: explain reveals the logical/physical plan; use to spot wide shuffles or scans.
df.filter(df.Count > 100).explain()


== Physical Plan ==
*(1) Project [Year#100, First Name#101 AS First_Name#131, County#102, Sex#103 AS Gender#133, Count#104]
+- *(1) Filter (isnotnull(Count#104) AND (Count#104 > 100))
   +- FileScan csv [Year#100,First Name#101,County#102,Sex#103,Count#104] Batched: false, DataFilters: [isnotnull(Count#104), (Count#104 > 100)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/opt/spark/data/nyc_health_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Count), GreaterThan(Count,100)], ReadSchema: struct<Year:int,First Name:string,County:string,Sex:string,Count:int>


